In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)
from sklearn import preprocessing
import pylab as pl
import scipy.optimize as opt
%matplotlib inline

# Load Data From CSV File

In [2]:
df = pd.read_csv('dataset.csv')
df.head()

,No.,material,Formula,CrystalSystem,bulk,shear,young,poisson,Hexp,best,y_cs,y_gap,y_den,y_csbg,y_csden,y_bgden
0,1,mp-66,Diamond,cubic,435.3,520.5,1116.5,0.07,96.0,H5,H1a,H2,H2,H2,H5,H2
1,2,mp-30148,BC2N,orthorhombic,361.0,422.7,912.1,0.08,76.0,H5,H2,H2,H2,H2,H2,H2
2,3,mp-629458,BC2N,orthorhombic,361.6,409.0,891.1,0.09,76.0,H5,H2,H1a,H2,H2,H2,H5
3,4,mp-1018649,c-BC5,trigonal,405.8,378.2,865.6,0.14,71.0,H2,H2,H4,H2,H2,H2,H2
4,5,mp-1639,BN,cubic,408.0,374.5,860.2,0.15,63.0,H2,H1a,H2,H2,H2,H5,H2


# Hardness Functions

In [4]:
def H1a(B,G,Y,v):
    return 0.1475*G

def H1b(B,G,Y,v):
    return 0.0607*Y

def H2(B,G,Y,v):
    return (0.1769*G)-2.899

def H3(B,G,Y,v):
    return 0.0635*Y

def H4(B,G,Y,v):
    return ((1-2*v)*B)/(6*(1+v))

def H5(B,G,Y,v):
    k = G/B
    return (2*np.power(k*k*G,0.585))-3

lookup={"H1a":H1a, "H1b":H1b, "H2":H2, "H3":H3, "H4":H4, "H5":H5}

# Gradient Boosting Classifier

In [5]:
X = df[['bulk','shear','young','poisson']] .values  #.astype(float)
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
y = df['best'].values

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.6, max_depth=1, random_state=0)
gbc.fit(X, y) 
y_gbc = gbc.predict(X)

In [8]:
H_gbc = np.zeros(143)
for i in range(0,142):
    function = lookup[y_gbc[i]]
    H_gbc[i] = function(df.bulk[i], df.shear[i], df.young[i], df.poisson[i])
error_gbc = np.abs(df.Hexp - H_gbc)
stddev_gbc = np.std(error_gbc)

In [9]:
from sklearn import metrics
print("All set Accuracy   = ", metrics.accuracy_score(y, gbc.predict(X)))
print("Mean abs error     = ", metrics.mean_absolute_error(df.Hexp, H_gbc))
print('Standard deviation = ', stddev_gbc)

All set Accuracy   =  0.9440559440559441
Mean abs error     =  1.1645333008969088
Standard deviation =  1.6456116287003573


In [10]:
dfmp = pd.read_csv('database_MP.csv')
dfmp.head()

,Material_ID,Formula,pretty_formula,KV,EV,GV,PV,KR,ER,GR,PR,bulk,shear,young,poisson,csyst,gap,density,y_cla
0,mp-47,C,C,435.44,1129.7,529.07,0.067615,435.42,1117.2,520.89,0.072372,435.43,524.98,1123.4,0.069988,hexagonal,3.3395,3.488249,H1b
1,mp-611426,C,C,435.78,1123.8,525.07,0.070181,435.77,1117.5,520.95,0.072589,435.77,523.01,1120.7,0.071384,hexagonal,4.5245,3.492429,H1b
2,mp-616440,C,C,435.56,1120.1,522.73,0.071390,435.55,1116.3,520.23,0.072856,435.55,521.48,1118.2,0.072122,hexagonal,4.3859,3.494842,H1b
3,mp-569567,C,C,435.56,1119.8,522.53,0.071507,435.55,1115.6,519.81,0.073100,435.55,521.17,1117.7,0.072303,trigonal,4.4083,3.489950,H2
4,mp-66,C,C,435.33,1120.0,522.80,0.071194,435.33,1113.0,518.22,0.073884,435.33,520.51,1116.5,0.072537,cubic,4.3387,3.495823,H2


In [11]:
X1 = dfmp[['bulk','shear','young','poisson']].values  #.astype(float)
y1_gbc = gbc.predict(X1)

In [13]:
y1_gbc.size
length = y1_gbc.size
length

11448

In [14]:
H1_gbc = np.zeros(length)
for i in range(0,length-1):
    function = lookup[y1_gbc[i]]
    H1_gbc[i] = function(dfmp.bulk[i], dfmp.shear[i], dfmp.young[i], dfmp.poisson[i])

Write data to CSV file

In [16]:
dfmp['y1_gbc'] = y1_gbc
dfmp['H1_gbc'] = H1_gbc

In [17]:
dfmp.head()

,Material_ID,Formula,pretty_formula,KV,EV,GV,PV,KR,ER,GR,...,bulk,shear,young,poisson,csyst,gap,density,y_cla,y1_gbc,H1_gbc
0,mp-47,C,C,435.44,1129.7,529.07,0.067615,435.42,1117.2,520.89,...,435.43,524.98,1123.4,0.069988,hexagonal,3.3395,3.488249,H1b,H2,89.969962
1,mp-611426,C,C,435.78,1123.8,525.07,0.070181,435.77,1117.5,520.95,...,435.77,523.01,1120.7,0.071384,hexagonal,4.5245,3.492429,H1b,H2,89.621469
2,mp-616440,C,C,435.56,1120.1,522.73,0.071390,435.55,1116.3,520.23,...,435.55,521.48,1118.2,0.072122,hexagonal,4.3859,3.494842,H1b,H2,89.350812
3,mp-569567,C,C,435.56,1119.8,522.53,0.071507,435.55,1115.6,519.81,...,435.55,521.17,1117.7,0.072303,trigonal,4.4083,3.489950,H2,H2,89.295973
4,mp-66,C,C,435.33,1120.0,522.80,0.071194,435.33,1113.0,518.22,...,435.33,520.51,1116.5,0.072537,cubic,4.3387,3.495823,H2,H2,89.179219


# Gradient Boosting Regressor

In [18]:
X = df[['bulk','shear','young','poisson']].values  #.astype(float)
y = df['Hexp'].values

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X, y)
H_gbr = reg.predict(X)

In [20]:
MAE_gbr = metrics.mean_absolute_error(df.Hexp, H_gbr)
error_gbr = np.abs(df.Hexp - H_gbr)
stddev_gbr = np.std(error_gbr)
print('Mean Absolu Error  = ', MAE_gbr)
print('Standard deviation = ', stddev_gbr)

Mean Absolu Error  =  0.9032806873399869
Standard deviation =  0.7293246456351304


In [21]:
X2 = dfmp[['bulk','shear','young','poisson']].values  #.astype(float)
H2_gbr = reg.predict(X2)

In [23]:
dfmp['H2_gbr'] = H2_gbr

In [24]:
dfmp.head()

,Material_ID,Formula,pretty_formula,KV,EV,GV,PV,KR,ER,GR,...,shear,young,poisson,csyst,gap,density,y_cla,y1_gbc,H1_gbc,H2_gbr
0,mp-47,C,C,435.44,1129.7,529.07,0.067615,435.42,1117.2,520.89,...,524.98,1123.4,0.069988,hexagonal,3.3395,3.488249,H1b,H2,89.969962,95.865939
1,mp-611426,C,C,435.78,1123.8,525.07,0.070181,435.77,1117.5,520.95,...,523.01,1120.7,0.071384,hexagonal,4.5245,3.492429,H1b,H2,89.621469,95.865939
2,mp-616440,C,C,435.56,1120.1,522.73,0.071390,435.55,1116.3,520.23,...,521.48,1118.2,0.072122,hexagonal,4.3859,3.494842,H1b,H2,89.350812,95.865939
3,mp-569567,C,C,435.56,1119.8,522.53,0.071507,435.55,1115.6,519.81,...,521.17,1117.7,0.072303,trigonal,4.4083,3.489950,H2,H2,89.295973,95.865939
4,mp-66,C,C,435.33,1120.0,522.80,0.071194,435.33,1113.0,518.22,...,520.51,1116.5,0.072537,cubic,4.3387,3.495823,H2,H2,89.179219,95.865939


# The Classic Calculator

In [25]:
dfmp.head()

,Material_ID,Formula,pretty_formula,KV,EV,GV,PV,KR,ER,GR,...,shear,young,poisson,csyst,gap,density,y_cla,y1_gbc,H1_gbc,H2_gbr
0,mp-47,C,C,435.44,1129.7,529.07,0.067615,435.42,1117.2,520.89,...,524.98,1123.4,0.069988,hexagonal,3.3395,3.488249,H1b,H2,89.969962,95.865939
1,mp-611426,C,C,435.78,1123.8,525.07,0.070181,435.77,1117.5,520.95,...,523.01,1120.7,0.071384,hexagonal,4.5245,3.492429,H1b,H2,89.621469,95.865939
2,mp-616440,C,C,435.56,1120.1,522.73,0.071390,435.55,1116.3,520.23,...,521.48,1118.2,0.072122,hexagonal,4.3859,3.494842,H1b,H2,89.350812,95.865939
3,mp-569567,C,C,435.56,1119.8,522.53,0.071507,435.55,1115.6,519.81,...,521.17,1117.7,0.072303,trigonal,4.4083,3.489950,H2,H2,89.295973,95.865939
4,mp-66,C,C,435.33,1120.0,522.80,0.071194,435.33,1113.0,518.22,...,520.51,1116.5,0.072537,cubic,4.3387,3.495823,H2,H2,89.179219,95.865939


In [27]:
y3_cla = dfmp['y_cla'].values

In [28]:
H3_cla = np.zeros(length)
for i in range(0,length-1):
    function = lookup[y3_cla[i]]
    H3_cla[i] = function(dfmp.bulk[i], dfmp.shear[i], dfmp.young[i], dfmp.poisson[i])

In [30]:
dfmp['H3_cla'] = H3_cla

In [31]:
dfmp.head()

,Material_ID,Formula,pretty_formula,KV,EV,GV,PV,KR,ER,GR,...,young,poisson,csyst,gap,density,y_cla,y1_gbc,H1_gbc,H2_gbr,H3_cla
0,mp-47,C,C,435.44,1129.7,529.07,0.067615,435.42,1117.2,520.89,...,1123.4,0.069988,hexagonal,3.3395,3.488249,H1b,H2,89.969962,95.865939,68.190380
1,mp-611426,C,C,435.78,1123.8,525.07,0.070181,435.77,1117.5,520.95,...,1120.7,0.071384,hexagonal,4.5245,3.492429,H1b,H2,89.621469,95.865939,68.026490
2,mp-616440,C,C,435.56,1120.1,522.73,0.071390,435.55,1116.3,520.23,...,1118.2,0.072122,hexagonal,4.3859,3.494842,H1b,H2,89.350812,95.865939,67.874740
3,mp-569567,C,C,435.56,1119.8,522.53,0.071507,435.55,1115.6,519.81,...,1117.7,0.072303,trigonal,4.4083,3.489950,H2,H2,89.295973,95.865939,89.295973
4,mp-66,C,C,435.33,1120.0,522.80,0.071194,435.33,1113.0,518.22,...,1116.5,0.072537,cubic,4.3387,3.495823,H2,H2,89.179219,95.865939,89.179219


# Print info to CSV and Excel files

In [32]:
dfmp.to_excel('prediction_all.xls', index=False)